In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
from socket import timeout
import http
import gc
import pandas as pd
import datetime
import csv
import numpy as np
import pyodbc
import time

In [7]:
import json
import pyodbc
#import numpy as np
server = 'masteksql.database.windows.net'
database = 'ProspectData'
username = 'yashpal'
password = 'Welcome498$'
driver= '{SQL Server}'

In [82]:
import json
import pyodbc
#import numpy as np
server = 'masteksql.database.windows.net'
database = 'ProspectData'
username = 'yashpal'
password = 'Welcome498$'
driver= '{SQL Server}'

cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

cursor.execute("Select ID,ProspectName,SearchResult from ProspectData WHERE ISPROCESSED='False'")
urllist=[]
bunch=[]
pid=[]
row = cursor.fetchone()
while row:
    bunch.append(str(row[2]))
    for prospect in bunch:
        data=json.loads(prospect)
    for each in data:
        urllist.append(each['url'])
        pid.append(row[0])
    row = cursor.fetchone()

d=[]
e=[]
for c in zip(pid,urllist):
    if c[1] not in d:
        d.append(c[0])
        e.append(c[1])
#return zip(e,d)
f=zip(d,e)

In [83]:
len(e)

2000

In [49]:
a=[1,2,3,4,5,6,7,8,9,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]

In [50]:
b=['a','b','c','d','e','f','g','h','i','a','b','c','d','e','f','g','h','i','a','b','c','d','e','f','g','h','i']

In [66]:
d

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i']

In [65]:
e

[1, 2, 3, 4, 5, 6, 7, 8, 9]

TypeError: object of type 'zip' has no len()

In [80]:
exclude=['twitter','glassdoor','youtube','wordpress','facebook','wikipedia','play.google','nasdaq']
allarcticles={}
logdcit={}
a=0
b=0
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8',
       'Connection': 'keep-alive'}
for g in f:
    print("link {}".format(a))
    if any (s in g[1] for s in exclude):
        print("excluded %s \n"%link)
        logdcit[a]=g[1] + "  **EXCLUDED"
    else:
        try:
            req = Request(g[1], headers=headers)
            with urlopen(req) as response:
                page = response.read()
            soup=BeautifulSoup(page, 'html.parser')
            ptag=soup.find_all('p')
            if len(ptag)<5:
                print("VERY LOW P TAGS, STUDY THIS - %s - %d" %(g[1], len(ptag)))
                logdcit[a]=g[1] + "  **VERY LOW P TAGS, STUDY THIS "
            text=[]
            for tag in ptag:
                text.append(tag.get_text())
            texta=str(' '.join(text)).replace('\n',' ')
            allarcticles[g[1]]=texta
            print("processed {} {} \n".format(g[1],b))
            b+=1
            logdcit[a]=g[1] + "  **PROCESSED"
        except http.client.IncompleteRead:
            print("INCOMPLETE READ - {}".format(g[1]))
            logdcit[a]=g[1] + "  **INCOMPLETE READ"
        except WindowsError as e:
            print("WINDOWS ERROR {} {}".format(e,g[1]))
            logdcit[a]=g[1] + "  **WINDOWS ERROR {}".format(e)
        except BaseException as error:
            print("SOME OTHER ERROR {}".format(link))
            logdcit[a]=g[1] + "  **SOME OTHER ERROR  {}".format(error)
            continue
        except timeout:
            print("LINK {} TIMED OUT \n".format(g[1]))
            timedout.append(g[1])
            logdcit[a]=g[1]  + "  **TIMED OUT"
    a+=1
  
filename=datetime.datetime.now().strftime("%d_%m_%H_%M_%S")
df1=pd.DataFrame.from_dict(logdcit,orient='index')
df1.to_csv("LogFile_"+datetime.datetime.now().strftime("%d_%m_%H_%M_%S")+".csv",sep=',',encoding='utf-8')
df2=pd.DataFrame.from_dict(allarcticles,orient='index')
df2.to_csv("SUMMARIES_"+datetime.datetime.now().strftime("%d_%m_%H_%M_%S")+".csv",sep=',',encoding='utf-8')
gc.collect    
    #try:    
    #    np.savetxt("accesserror.csv", accesserror, delimiter=",", fmt='%s')
    #except:
    #    print("cant save accesserror using np")


link 0
processed https://www.theguardian.com/business/2018/jun/19/capita-awarded-mod-contract-despite-having-highest-risk-rating 0 

link 1
excluded https://www.prnewswire.com/news-releases/prometic-announces-the-publication-of-pbi-4050s-antifibrotic-activity-in-liver-diseases-in-journal-of-pharmacology-and-experimental-therapeutics-300695315.html 

link 2
excluded https://www.prnewswire.com/news-releases/prometic-announces-the-publication-of-pbi-4050s-antifibrotic-activity-in-liver-diseases-in-journal-of-pharmacology-and-experimental-therapeutics-300695315.html 

link 3
processed http://dccapture.com/2018/07/25/nhs-contract-with-capita-could-have-put-patients-at-risk/ 1 

link 4
processed http://directorsclub.news/2018/07/04/capita-secures-contract-extension-with-npower/ 2 

link 5
processed https://www.payscale.com/research/IN/Employer=Capita_india/Salary 3 

link 6
WINDOWS ERROR HTTP Error 404: Not Found http://www.premiumgazette.com/nhs-contract-with-capita-could-have-put-patients-

processed https://www.ceicdata.com/en/seychelles/government-revenue-expenditure-and-finance/sc-net-acquisition-of-financial-assets--of-gdp 37 

link 55
processed https://www.business-standard.com/article/economy-policy/lic-s-acquisition-of-51-in-idbi-bank-a-win-win-situation-for-both-goyal-118080200039_1.html 38 

link 56
processed https://www.business-case-analysis.com/capital-equipment.html 39 

link 57
processed https://www.hilcocapital.com/ 40 

link 58
processed https://www.thebalancesmb.com/how-a-business-owner-s-capital-account-works-398172 41 

link 59
VERY LOW P TAGS, STUDY THIS - http://singaporestockmarketnews.blogspot.com/2018/06/capitaland-commercial-trust-gallileo.html - 2
processed http://singaporestockmarketnews.blogspot.com/2018/06/capitaland-commercial-trust-gallileo.html 42 

link 60
excluded https://www.prnewswire.com/news-releases/prometic-announces-the-publication-of-pbi-4050s-antifibrotic-activity-in-liver-diseases-in-journal-of-pharmacology-and-experimental-ther

WINDOWS ERROR HTTP Error 403: Forbidden https://www.usnews.com/news/world/articles/2018-08-01/sweden-to-sign-1-billion-patriot-missile-deal-this-week-report
link 111
processed http://www.oxfordhandbooks.com/view/10.1093/oxfordhb/9780199945474.001.0001/oxfordhb-9780199945474-e-017 83 

link 112
processed https://www.sabreakingnews.co.za/2018/06/16/xhaka-agrees-new-arsenal-contract/ 84 

link 113
processed https://money.cnn.com/2018/06/19/investing/market-safety-stocks-tariffs-trade-war/index.html 85 

link 114
excluded https://www.prnewswire.com/news-releases/prometic-announces-the-publication-of-pbi-4050s-antifibrotic-activity-in-liver-diseases-in-journal-of-pharmacology-and-experimental-therapeutics-300695315.html 

link 115
processed https://www.thebalancesmb.com/five-contract-templates-for-freelancers-1360292 86 

link 116
excluded https://www.prnewswire.com/news-releases/prometic-announces-the-publication-of-pbi-4050s-antifibrotic-activity-in-liver-diseases-in-journal-of-pharmacolo

processed https://www.marketwatch.com/story/aigs-stock-falls-after-reporting-what-analyst-called-weak-results-2018-08-03 123 

link 164
processed https://www.business-standard.com/article/pti-stories/jet-airways-shares-hit-52-week-low-on-weak-earnings-118052400390_1.html 124 

link 165
processed https://www.fool.com/investing/2018/06/20/weak-bookings-weak-prospects-wreck-saic-stock.aspx 125 

link 166
processed https://www.marketwatch.com/story/ftse-100-recovering-from-1-week-low-but-glencore-sinks-on-doj-subpoena-2018-07-03 126 

link 167
processed https://247wallst.com/technology-3/2018/08/15/why-casa-systems-is-wednesdays-big-earnings-loser/ 127 

link 168
processed https://www.fool.com/investing/2018/08/21/why-coty-inc-stock-was-falling-today.aspx 128 

link 169
excluded https://www.prnewswire.com/news-releases/prometic-announces-the-publication-of-pbi-4050s-antifibrotic-activity-in-liver-diseases-in-journal-of-pharmacology-and-experimental-therapeutics-300695315.html 

link 170
WI

processed http://www.worldusheadlines.com/2018/07/18/british-outsourcer-capitas-cfo/ 154 

link 217
excluded https://www.prnewswire.com/news-releases/prometic-announces-the-publication-of-pbi-4050s-antifibrotic-activity-in-liver-diseases-in-journal-of-pharmacology-and-experimental-therapeutics-300695315.html 

link 218
VERY LOW P TAGS, STUDY THIS - https://www.bloomberg.com/research/stocks/private/snapshot.asp?privcapId=5491410 - 2
processed https://www.bloomberg.com/research/stocks/private/snapshot.asp?privcapId=5491410 155 

link 219
processed https://tradersnews.com/2018/06/19/capita-sells-supplier-assessment-services-for-160m/ 156 

link 220
excluded https://www.prnewswire.com/news-releases/prometic-announces-the-publication-of-pbi-4050s-antifibrotic-activity-in-liver-diseases-in-journal-of-pharmacology-and-experimental-therapeutics-300695315.html 

link 221
excluded https://www.prnewswire.com/news-releases/prometic-announces-the-publication-of-pbi-4050s-antifibrotic-activity-in-li

...

processed https://www.verywellhealth.com/deep-tissue-massage-89738 1385 

link 1919
processed https://www.omaha.com/ 1386 

link 1920
VERY LOW P TAGS, STUDY THIS - https://www.hellopeter.com/spec-savers-sa - 0
processed https://www.hellopeter.com/spec-savers-sa 1387 

link 1921
VERY LOW P TAGS, STUDY THIS - https://www.payscale.com/research/UK/Job=Optical_Assistant/Salary/9aba04da/Specsavers - 3
processed https://www.payscale.com/research/UK/Job=Optical_Assistant/Salary/9aba04da/Specsavers 1388 

link 1922
excluded https://www.prnewswire.com/news-releases/prometic-announces-the-publication-of-pbi-4050s-antifibrotic-activity-in-liver-diseases-in-journal-of-pharmacology-and-experimental-therapeutics-300695315.html 

link 1923
VERY LOW P TAGS, STUDY THIS - https://www.payscale.com/research/IE/Job=Optical_Assistant/Salary - 3
processed https://www.payscale.com/research/IE/Job=Optical_Assistant/Salary 1389 

link 1924
processed https://www.thebalancecareers.com/customer-service-cover-letter

processed https://www.payscale.com/research/UK/Job=Retail_Store_Manager/Salary 1418 

link 1973
excluded https://www.prnewswire.com/news-releases/prometic-announces-the-publication-of-pbi-4050s-antifibrotic-activity-in-liver-diseases-in-journal-of-pharmacology-and-experimental-therapeutics-300695315.html 

link 1974
processed https://www.payscale.com/research/AU/Job=Retail_Store_Manager/Salary 1419 

link 1975
processed https://www.essexcricket.org.uk/2018/06/27/match-report-essex-v-somerset-2/ 1420 

link 1976
processed https://www.middlesexccc.com/fixtures/results/2018/06/20/leicestershire-v-middlesex 1421 

link 1977
processed https://www.tripadvisor.com/Restaurant_Review-g186337-d6877795-Reviews-TJ_Hughes-Liverpool_Merseyside_England.html 1422 

link 1978
processed https://www.essexcricket.org.uk/2018/08/21/match-report-somerset-v-essex-2/ 1423 

link 1979
excluded https://www.prnewswire.com/news-releases/prometic-announces-the-publication-of-pbi-4050s-antifibrotic-activity-in-live

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


VERY LOW P TAGS, STUDY THIS - http://pubdocs.worldbank.org/en/294331530217033360/ESF-GN5-June-2018.pdf - 0
processed http://pubdocs.worldbank.org/en/294331530217033360/ESF-GN5-June-2018.pdf 1429 

link 1990
processed https://www.sciencedirect.com/science/article/pii/S0945053X18300544 1430 

link 1991
processed http://inpublic.globenewswire.com/2018/07/09/Galapagos+announces+design+for+PINTA+Phase+2+trial+with+GLPG1205+in+IPF+HUG2203720.html 1431 

link 1992
processed http://investors.kadmon.com/recent-press-releases/2018 1432 

link 1993
processed http://www.4-traders.com/FIBROGEN-INC-18537026/news/FibroGen-to-Report-Second-Quarter-2018-Financial-Results-on-Tuesday-August-7-2018-27039542/ 1433 

link 1994
processed http://inpublic.globenewswire.com/2018/08/02/Productive+first+half+year+at+Galapagos+HUG2208982.html 1434 

link 1995
processed https://www.marketwatch.com/press-release/prometic-reports-second-quarter-2018-financial-results-and-highlights-2018-08-14 1435 

link 1996
process

<function gc.collect>